# % в просмотре и % в вещании жанров на ТВ
Пример расчета просмотра и показа жанров на ТВ

## Описание задачи и условий расчета
- Регион: Россия 0+
- Период: 2024 год
- ЦА: Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Программы: группировка по жанрам
- Статистики:
    - % в просмотре (спрос) - % просмотра определённого жанра от всего времени просмотра программ всех жанров
    - % в вещании (предложение) - % времени вещания определённого жанра от общего времени вещания программ всех жанров

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Время просмотра и длительность
Рассчитаем время просмотра и длительность (отдельно по жанрам и тотал)

### По жанрам
#### Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2024-01-01', '2024-12-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: без учета рекламных каналов
company_filter = 'tvThematicId != 19'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов (группировок)
slices = [
    'programTypeName', #Программа жанр
]

# Указываем список статистик для расчета
statistics = [
    'ATVSum', 
    'DurationSum'
]

# Задаем условия сортировки: жанр (по алф.)
sortings = {'programTypeName':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "PROGRAM", #Тип события - Программы
}

#### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_crosstab = mtask.wait_task(mtask.send_crosstab_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_crosstab))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

### Тотал
#### Формирование задания
Зададим условия расчета

In [ ]:
# убираем из срезов жанры
slices = []

# Очищаем условие сортировки
sortings = None

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_crosstab = mtask.wait_task(mtask.send_crosstab_task(task_json))

# Получаем результат
df_totals = mtask.result2table(mtask.get_result(task_crosstab))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df_totals = df_totals[slices+statistics]

# Добавляем колонку для корректного объединения с таблицей жанров
df_totals.insert(0, 'programTypeName', 'Total')
df_totals

### Объединяем таблицы

In [ ]:
# Добавляем тотал вниз таблицы с жанрами
df = pd.concat([df, df_totals], ignore_index=True)
df

## Расчет процентов
Рассчитаем проценты по жанрам от тотал показателей

In [ ]:
df.set_index('programTypeName', inplace=True)

# Рассчитаем спрос
df['% в просмотре'] = df['ATVSum']/df.loc['Total','ATVSum']*100

# Рассчитаем предложение
df['% в вещании'] = df['DurationSum']/df.loc['Total','DurationSum']*100

In [ ]:
df

In [ ]:
# Округляем значения в таблице до 2 знаков
df = df.round(2)

In [ ]:
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('06_supply_and_demand_TTV')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)